Part 1: Markov chains

Markov chains are crucial to Markov Chain Monte Carlo (as you would expect). So, we'll spend a few minutes here practicing with them and looking at a few properties.

As noted in the lecture, the state of a Markov chain is dependent on only its most recent previous state. This is made clear by this factorization:

$$P(x_t|x_{t-1}, x_{t-2}, ..., x_{0}) = P(x_t | x_{t-1})$$

This is known as the "memorylessness" property.

Additionally, for use in MCMC, our Markov chains need to have the ergodic property. In other words, they must be irreducible, time-homogenous, aperiodic, and have a stationary distribution.

(Irreducible: Every state is reachable from every other state.)

(Time-homogenous: The transition matrix does not depend on the sampling time step.)

(Aperiodic: There are (generally) no constraints on path length between states.)

The Markov chain transition matrix must also be a "stochastic matrix": all values are nonnegative, and every row sums to one.

1) Implement a predicate function to determine if a given matrix is a stochastic matrix. You may assume the input is a 2-dimensional square `numpy` array.

2) Given a finite square transition matrix, how could you verify that it is irreducible? Think algorithmically. (Note that I am asking for a plain-English explanation. If you wish, you can implement a function for this but it is not required.)

3) Implement a function that computes one step of a Markov chain. It should take as input a sample vector and a transition matrix, and return a new probability vector. You may assume the inputs are semantically valid for the Markov chain (i.e. that the sample vector sums to one, and that the transition matrix is ergodic).

You can begin with this boilerplate:

```python
def step_markov_chain(sample, transition_matrix):
    # do work
    return new_sample
```

4) Use your function to find the stationary distribution of the following transition matrix T.

Remember that the initial sample (the guess) must be a valid probability assignment (sum to 1).

In [ ]:
import numpy as np

T = np.array([[0.2, 0.8], [0.1, 0.9]])

x = pass # DO THIS
print(x)
for i in range(10):
    x = step_markov_chain(x, T)
    print(x)

5) In your own words, describe what the stationary distribution is.

6) Experiment with three different starting guesses for the first sample.

7) Why is the first sample important?

8) Create three different transition matrices and find their stationary distributions. You may need to experiment with your matrices to make sure that the stationary distribution exists. In your own words, explain why your transition matrices are irreducible.

Part 2: Markov Chain Monte Carlo

In [ ]:
# Preliminaries

%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

sns.set_style('white')
sns.set_context('talk')

np.random.seed(42)

Let's implement a Metropolis MCMC sampler, then compare it to one in a standard statistical library.

First, we generate some data: 100 points from a normal distribution with mean 1:

In [ ]:
data = np.random.normal(loc=1, size=100)

We can plot a histogram of those points:

In [ ]:
ax = plt.subplot()
sns.distplot(data, kde=False, ax=ax)
_ = ax.set(title='Histogram of observed data', xlabel='x', ylabel='# observations');

We need to choose a model. We already know the data generating process is a normal distribution with mean 1 and variance 0. For the sake of argument, we will try to infer the mean.

Because we are inferring the mean, the posterior distribution of our model will be our estimate for $\mu$:

$$\mu \sim \mathcal{N}(0, 1)$$

$$x | \mu \sim \mathcal{N}(x; \mu, 1)$$

In words, this says:

The mean has a prior distribution of $\mathcal{N}(0, 1)$ (which we chose arbitrarily).

The data, conditioned on the (unknown) mean, is distributed according to a normal distribution (which is parameterized by the data, the unknown mean, and a $\sigma$ of 1).


We will now build a Metropolis sampler for inferring one unknown parameter.

Recall the general MCMC algorithm:
0. Begin with an initial point.
1. Create a new proposal point to "jump" to.
2. Evaluate its likelihood and create an acceptance ratio.
3. Accept the proposal according to that acceptance ratio.
5. Goto 2.

Fill in the following code.

In [ ]:
# make_p is a utility function for making a version of our model with the proposed mu.
# using this separates the model from the sampler.
def make_p(mu):
    return norm(mu, 1)

def metropolis_sampler(initial_point, data, p_maker, n_samples, prior, proposal_stddev):
    posterior = [initial_point]
    
    # the result of p_maker should support the 'pdf' function
    
    n_dims = len(initial_point) # the number of dimensions
    current = initial_point
    
    for i in range(n_samples):
        # Metropolis creates proposals from a normal distribution:
        proposal = norm.rvs(current, proposal_stddev, n_dims)
        
        # Compute likelihood by multiplying probabilities of each point:
        likelihood_current = p_maker(current).pdf(data).prod()
        likelihood_proposal = pass # DO THIS
        
        # Compute the prior probability of current and proposed value:
        prior_current = prior.pdf(current)
        prior_proposal = pass # DO THIS
        
        p_current = likelihood_current * prior_current
        p_proposal = pass # DO THIS
        
        # Accept proposal?
        p_accept = pass # DO THIS
        
        # Usually would include prior probability, which we neglect here for simplicity
        accept = np.random.rand() < p_accept
        
        if accept:
            # Update position
            current = proposal
    
        posterior.append(current)
    return posterior
            

Use it to compute the expectation of $\mu$ in this model with 1.1 as an initial guess:

```python
metropolis_sampler([1.1], data, make_p, 100, norm(), 1)
```

Use the points 2, -1, 0, 1, 2 as initial guesses and compute the expected means.

Experiment with different numbers of samples. Conjecture a relationship between the initial guess and the convergence rate.



Try to find an initial guess that does not seem to converge after 1000 samples. (Note that you may have to avoid values that are too extreme, due to numerical overflow issues. [If you wish, you can update the sampler to operate in log-probability space to fix this.])

Experiment with different proposal standard deviation values. Conjecture a relationship between the stddev and the convergence rate.


Now we'll create a plot of our traces. This plot shows the value of each sample.

In [ ]:
traces = metropolis_sampler([1.1], data, make_p, 1000, norm(), 0.1)
fig, ax = plt.subplots()
ax.plot([a[0] for a in traces])
_ = ax.set(xlabel='sample', ylabel='mu');

Plot traces of three other initial guesses and three other proposal standard deviations. Give explanations for why the plots look the way they do.

"Burn-in" is a term for the initial samples that we want to discard. Why would we want to discard them? Can you think of a way to determine when we should stop the burn-in process?

To wrap up, let's use PyMC3 to compare to our handwritten sampler.

In [ ]:
import pymc3 as pm

with pm.Model():
    mu = pm.Normal('mu', 1, 1)
    sigma = 1.
    returns = pm.Normal('returns', mu=mu, sd=sigma, observed=data)
    
    step = pm.Metropolis()
    pymc3_traces = pm.sample(15000, step)
    
assert len(traces) > 1000, 'run the sampler with high n_samples'

sns.distplot(pymc3_traces[2000:]['mu'], label='PyMC3 sampler');
sns.distplot(traces[500:], label='Hand-written sampler');
plt.legend();

Verify through the above that our sampler matches that of PyMC3's Metropolis sampler.